In [ ]:
import numpy as np
import os
import scipy.signal as signal
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew
import pickle
import pandas as pd
import re
import warnings
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
from sklearn.decomposition import PCA
warnings.filterwarnings('ignore')  # 忽略警告

# 设置matplotlib支持中文
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

class MotionDetection:
    def __init__(self):
        self.model = None
        self.threshold = None
        self.feature_names = ["方差", "平均幅度变化", "幅度标准差", "峰度", "偏度", "前后半段差异", "最大幅度变化", "信号能量"]
        
    def load_csi_data(self, file_path, visualize=False):
        """
        加载CSI数据文件 (CSV格式)
        """
        try:
            # 读取CSV数据
            df = pd.read_csv(file_path)
            
            # 检查必要的列是否存在
            if 'type' not in df.columns or 'data' not in df.columns:
                print(f"文件格式错误: {file_path}，缺少必要的列")
                return None
            
            # 提取CSI数据
            csi_data = []
            timestamps = []  # 记录时间戳用于可视化
            
            for index, row in df.iterrows():
                if row['type'] == 'CSI_DATA':
                    try:
                        # 获取时间戳
                        if 'local_timestamp' in row:
                            timestamps.append(row['local_timestamp'])
                        else:
                            timestamps.append(index)  # 如果没有时间戳，使用索引
                            
                        # 解析data字段中的CSI数据
                        data_str = str(row['data'])
                        # 去除引号和方括号
                        data_str = data_str.strip('"[]')
                        # 解析为数值列表
                        values = []
                        for val in data_str.split(','):
                            try:
                                values.append(int(val.strip()))
                            except ValueError:
                                # 跳过无法转换为整数的值
                                pass
                        
                        # 将数据转换为复数形式 (I和Q分量)
                        complex_values = []
                        for i in range(0, len(values), 2):
                            if i+1 < len(values):
                                # 转换为复数: I + jQ
                                complex_values.append(complex(values[i], values[i+1]))
                        
                        if len(complex_values) > 0:
                            csi_data.append(complex_values)
                    except Exception as e:
                        print(f"解析CSI数据行错误: {str(e)}")
                        continue
            
            if len(csi_data) == 0:
                print(f"无法从文件中提取CSI数据: {file_path}")
                return None
            
            csi_array = np.array(csi_data)
            
            # 如果需要可视化，绘制原始CSI数据图
            if visualize and len(csi_array) > 0:
                filename = os.path.basename(file_path)
                self._visualize_raw_csi(csi_array, timestamps, filename)
                
            return csi_array
            
        except Exception as e:
            print(f"无法加载文件: {file_path}")
            print(f"错误: {str(e)}")
            return None
    
    def _visualize_raw_csi(self, csi_data, timestamps, filename):
        """可视化原始CSI数据"""
        try:
            # 计算CSI幅值
            amplitude = np.abs(csi_data)
            
            # 创建一个大图
            plt.figure(figsize=(15, 10))
            
            # 设置网格布局
            gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1.5])
            
            # 1. 显示CSI幅值热图 - 子载波随时间的变化
            ax1 = plt.subplot(gs[0, :])
            im = ax1.imshow(amplitude.T, aspect='auto', interpolation='none', 
                           cmap='viridis', norm=LogNorm())
            plt.colorbar(im, ax=ax1, label='幅值')
            ax1.set_title(f'CSI幅值热图 - {filename}')
            ax1.set_xlabel('包索引')
            ax1.set_ylabel('子载波索引')
            
            # 2. 时间序列图 - 平均幅值随时间变化
            ax2 = plt.subplot(gs[1, 0])
            mean_amp = np.mean(amplitude, axis=1)  # 计算每个时间点的平均幅值
            ax2.plot(mean_amp, color='blue', linewidth=2)
            ax2.set_title('平均幅值随时间变化')
            ax2.set_xlabel('包索引')
            ax2.set_ylabel('平均幅值')
            ax2.grid(True, linestyle='--', alpha=0.7)
            
            # 3. 子载波平均幅值图
            ax3 = plt.subplot(gs[1, 1])
            mean_subcarr = np.mean(amplitude, axis=0)  # 计算每个子载波的平均幅值
            ax3.plot(mean_subcarr, color='green', linewidth=2)
            ax3.set_title('各子载波平均幅值')
            ax3.set_xlabel('子载波索引')
            ax3.set_ylabel('平均幅值')
            ax3.grid(True, linestyle='--', alpha=0.7)
            
            plt.tight_layout()
            viz_dir = 'motion_output/visualization'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, f'raw_csi_{filename}.png'))
            plt.close()
            print(f"已保存原始CSI可视化: raw_csi_{filename}.png")
        except Exception as e:
            print(f"可视化原始CSI数据失败: {str(e)}")
    
    def preprocess_csi(self, csi_data, visualize=False, filename='unknown'):
        """
        预处理CSI数据
        """
        if csi_data is None or len(csi_data) == 0:
            return None
        
        try:
            # 取CSI数据的幅值
            amplitude = np.abs(csi_data)
            
            # 检查数据是否包含NaN或无穷大
            if np.isnan(amplitude).any() or np.isinf(amplitude).any():
                print("警告: 数据包含NaN或无穷大值，将被替换")
                amplitude = np.nan_to_num(amplitude, nan=0.0, posinf=0.0, neginf=0.0)
            
            # 检查数据的一致性
            if amplitude.size == 0 or amplitude.ndim < 2:
                print("警告: 数据格式不一致或为空")
                return None
            
            # 应用中值滤波器去除噪声
            try:
                filtered_data = signal.medfilt(amplitude, kernel_size=3)
            except Exception as e:
                print(f"应用中值滤波器失败: {str(e)}")
                filtered_data = amplitude  # 如果滤波失败，使用原始数据
            
            # 归一化
            try:
                mean_data = np.mean(filtered_data, axis=0, keepdims=True)
                std_data = np.std(filtered_data, axis=0, keepdims=True)
                # 避免除以零或很小的数
                normalized_data = (filtered_data - mean_data) / (std_data + 1e-6)
                
                # 检查归一化结果
                if np.isnan(normalized_data).any() or np.isinf(normalized_data).any():
                    print("警告: 归一化后数据包含NaN或无穷大值，将被替换")
                    normalized_data = np.nan_to_num(normalized_data, nan=0.0, posinf=0.0, neginf=0.0)
            except Exception as e:
                print(f"归一化失败: {str(e)}")
                normalized_data = filtered_data  # 如果归一化失败，使用过滤后的数据
            
            # 可视化预处理过程
            if visualize:
                self._visualize_preprocessing(amplitude, filtered_data, normalized_data, filename)
            
            return normalized_data
        except Exception as e:
            print(f"预处理CSI数据失败: {str(e)}")
            return None
    
    def _visualize_preprocessing(self, raw_data, filtered_data, normalized_data, filename):
        """可视化预处理步骤"""
        try:
            plt.figure(figsize=(15, 12))
            
            # 1. 原始数据
            plt.subplot(3, 1, 1)
            plt.imshow(raw_data.T, aspect='auto', cmap='viridis')
            plt.colorbar(label='幅值')
            plt.title('原始CSI幅值')
            plt.xlabel('包索引')
            plt.ylabel('子载波索引')
            
            # 2. 滤波后数据
            plt.subplot(3, 1, 2)
            plt.imshow(filtered_data.T, aspect='auto', cmap='viridis')
            plt.colorbar(label='幅值')
            plt.title('中值滤波后CSI幅值')
            plt.xlabel('包索引')
            plt.ylabel('子载波索引')
            
            # 3. 归一化后数据
            plt.subplot(3, 1, 3)
            plt.imshow(normalized_data.T, aspect='auto', cmap='seismic', vmin=-3, vmax=3)
            plt.colorbar(label='标准化幅值')
            plt.title('归一化后CSI幅值')
            plt.xlabel('包索引')
            plt.ylabel('子载波索引')
            
            plt.tight_layout()
            viz_dir = 'motion_output/visualization'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, f'preprocess_{filename}.png'))
            plt.close()
            print(f"已保存预处理可视化: preprocess_{filename}.png")
        except Exception as e:
            print(f"可视化预处理步骤失败: {str(e)}")
    
    def extract_features(self, preprocessed_data, visualize=False, filename='unknown'):
        """
        从预处理的CSI数据中提取特征
        """
        if preprocessed_data is None:
            return None
        
        try:
            feature_values = {}
            
            # 1. 计算方差 - 运动通常会导致更高的方差
            variance = np.var(preprocessed_data, axis=0)
            feature_values["方差"] = np.mean(variance)
            
            # 2. 计算各子载波的平均幅度变化
            diff = np.abs(np.diff(preprocessed_data, axis=0))
            mean_diff = np.mean(diff, axis=0)
            feature_values["平均幅度变化"] = np.mean(mean_diff)
            
            # 3. 计算各子载波的幅度标准差
            std_amp = np.std(preprocessed_data, axis=0)
            feature_values["幅度标准差"] = np.mean(std_amp)
            
            # 4. 计算各子载波的峰度
            try:
                kurt = kurtosis(preprocessed_data, axis=0)
                feature_values["峰度"] = np.mean(kurt)
            except Exception as e:
                print(f"计算峰度失败: {str(e)}")
                feature_values["峰度"] = 0.0
            
            # 5. 计算各子载波的偏度
            try:
                skewness = skew(preprocessed_data, axis=0)
                feature_values["偏度"] = np.mean(skewness)
            except Exception as e:
                print(f"计算偏度失败: {str(e)}")
                feature_values["偏度"] = 0.0
            
            # 6. 计算前半段和后半段的平均差异
            n_samples = preprocessed_data.shape[0]
            first_half = preprocessed_data[:n_samples//2]
            second_half = preprocessed_data[n_samples//2:]
            half_diff = np.mean(np.abs(np.mean(first_half, axis=0) - np.mean(second_half, axis=0)))
            feature_values["前后半段差异"] = half_diff
            
            # 7. 计算最大幅度变化
            max_diff = np.max(diff, axis=0)
            feature_values["最大幅度变化"] = np.mean(max_diff)
            
            # 8. 计算信号能量
            energy = np.sum(np.square(preprocessed_data), axis=0)
            feature_values["信号能量"] = np.mean(energy)
            
            # 转换为numpy数组
            features = np.array([v for v in feature_values.values()])
            
            # 检查特征值是否包含NaN或无穷大
            if np.isnan(features).any() or np.isinf(features).any():
                print("警告: 特征包含NaN或无穷大值，将被替换")
                features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
            
            # 可视化提取的特征
            if visualize:
                self._visualize_features(feature_values, filename)
            
            return features
        except Exception as e:
            print(f"提取特征失败: {str(e)}")
            return None
    
    def _visualize_features(self, feature_values, filename):
        """可视化提取的特征"""
        try:
            plt.figure(figsize=(12, 8))
            
            # 绘制所有特征的雷达图
            features = list(feature_values.keys())
            values = list(feature_values.values())
            
            # 标准化特征值以适合雷达图
            max_val = max(values)
            if max_val > 0:
                norm_values = [v/max_val for v in values]
            else:
                norm_values = values
                
            # 准备雷达图数据
            angles = np.linspace(0, 2*np.pi, len(features), endpoint=False).tolist()
            values = norm_values + [norm_values[0]]  # 闭合雷达图
            angles = angles + [angles[0]]  # 闭合雷达图
            features = features + [features[0]]  # 闭合标签
            
            # 绘制雷达图
            ax = plt.subplot(111, polar=True)
            ax.plot(angles, values, 'o-', linewidth=2)
            ax.fill(angles, values, alpha=0.25)
            ax.set_thetagrids(np.degrees(angles[:-1]), features[:-1])
            
            plt.title(f'特征雷达图 - {filename}')
            
            # 添加原始特征值文本框
            textstr = '\n'.join([f'{k}: {v:.4f}' for k, v in feature_values.items()])
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            plt.figtext(0.15, 0.05, textstr, fontsize=10, bbox=props)
            
            plt.tight_layout()
            viz_dir = 'motion_output/visualization'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, f'features_{filename}.png'))
            plt.close()
            print(f"已保存特征可视化: features_{filename}.png")
        except Exception as e:
            print(f"可视化特征失败: {str(e)}")
    
    def train(self, motion_dir, static_dir, visualize=True):
        """
        使用标记数据训练模型
        """
        print("\n==== 开始模型训练 ====")
        print("开始加载运动和静止数据...")
        
        # 加载运动数据
        motion_features = []
        motion_files = os.listdir(motion_dir)
        print(f"发现 {len(motion_files)} 个运动数据文件")
        
        # 选择部分文件进行可视化展示
        vis_sample_size = min(3, len(motion_files))
        vis_motion_files = np.random.choice(motion_files, size=vis_sample_size, replace=False)
        
        for file in motion_files:
            if file.endswith('.csv'):
                file_path = os.path.join(motion_dir, file)
                print(f"处理运动文件: {file}")
                
                # 决定是否可视化
                vis_this_file = file in vis_motion_files and visualize
                
                csi_data = self.load_csi_data(file_path, visualize=vis_this_file)
                
                if csi_data is not None:
                    preprocessed_data = self.preprocess_csi(csi_data, visualize=vis_this_file, filename=file)
                    if preprocessed_data is not None:
                        features = self.extract_features(preprocessed_data, visualize=vis_this_file, filename=file)
                        if features is not None:
                            motion_features.append(features)
                            print(f"成功提取特征: {file}")
                        else:
                            print(f"提取特征失败: {file}")
                    else:
                        print(f"预处理失败: {file}")
                else:
                    print(f"加载数据失败: {file}")
        
        # 加载静止数据
        static_features = []
        static_files = os.listdir(static_dir)
        print(f"发现 {len(static_files)} 个静止数据文件")
        
        # 选择部分文件进行可视化展示
        vis_sample_size = min(3, len(static_files))
        vis_static_files = np.random.choice(static_files, size=vis_sample_size, replace=False)
        
        for file in static_files:
            if file.endswith('.csv'):
                file_path = os.path.join(static_dir, file)
                print(f"处理静止文件: {file}")
                
                # 决定是否可视化
                vis_this_file = file in vis_static_files and visualize
                
                csi_data = self.load_csi_data(file_path, visualize=vis_this_file)
                
                if csi_data is not None:
                    preprocessed_data = self.preprocess_csi(csi_data, visualize=vis_this_file, filename=file)
                    if preprocessed_data is not None:
                        features = self.extract_features(preprocessed_data, visualize=vis_this_file, filename=file)
                        if features is not None:
                            static_features.append(features)
                            print(f"成功提取特征: {file}")
                        else:
                            print(f"提取特征失败: {file}")
                    else:
                        print(f"预处理失败: {file}")
                else:
                    print(f"加载数据失败: {file}")
        
        # 检查是否有足够的数据
        if len(motion_features) == 0 or len(static_features) == 0:
            print("错误: 没有提取到足够的特征数据!")
            return
            
        # 转换为numpy数组
        motion_features = np.array(motion_features)
        static_features = np.array(static_features)
        
        print(f"\n==== 特征分析 ====")
        print(f"运动特征样本数: {len(motion_features)}")
        print(f"静止特征样本数: {len(static_features)}")
        
        # 可视化所有特征的分布对比
        if visualize:
            self._visualize_all_features_distribution(motion_features, static_features)
        
        # 寻找最具辨别力的特征
        motion_mean = np.mean(motion_features, axis=0)
        static_mean = np.mean(static_features, axis=0)
        feature_diff = np.abs(motion_mean - static_mean)
        
        # 检查特征差异是否为NaN
        if np.isnan(feature_diff).any():
            print("警告: 特征差异包含NaN值")
            feature_diff = np.nan_to_num(feature_diff, nan=0.0)
        
        best_feature_idx = np.argmax(feature_diff)
        
        # 使用最佳特征计算阈值
        motion_values = motion_features[:, best_feature_idx]
        static_values = static_features[:, best_feature_idx]
        
        # 检查值是否包含NaN
        if np.isnan(motion_values).any() or np.isnan(static_values).any():
            print("警告: 特征值包含NaN值，将被替换")
            motion_values = np.nan_to_num(motion_values, nan=0.0)
            static_values = np.nan_to_num(static_values, nan=0.0)
        
        # 阈值选择: 运动与静止特征的中点
        motion_mean_val = np.mean(motion_values)
        static_mean_val = np.mean(static_values)
        
        print(f"\n==== 特征分析结果 ====")
        # 输出所有特征的均值比较
        print("各特征均值比较:")
        for i, name in enumerate(self.feature_names):
            print(f"{name}: 运动={motion_mean[i]:.4f}, 静止={static_mean[i]:.4f}, 差异={feature_diff[i]:.4f}")
        
        print(f"\n最具区分性的特征: {self.feature_names[best_feature_idx]}")
        print(f"运动样本该特征平均值: {motion_mean_val:.4f}")
        print(f"静止样本该特征平均值: {static_mean_val:.4f}")
        
        self.threshold = (motion_mean_val + static_mean_val) / 2
        self.best_feature_idx = best_feature_idx
        
        print(f"计算得到的阈值: {self.threshold:.4f}")
        
        # 得到预测结果
        motion_predictions = (motion_values > self.threshold).astype(int)
        static_predictions = (static_values > self.threshold).astype(int)
        
        # 评估训练集准确度
        # 注意: 需要考虑特征与标签的关系，可能需要反转预测结果
        motion_correct = np.sum(motion_predictions == 1)
        static_correct = np.sum(static_predictions == 0)
        
        motion_accuracy = motion_correct / len(motion_predictions) if len(motion_predictions) > 0 else 0
        static_accuracy = static_correct / len(static_predictions) if len(static_predictions) > 0 else 0
        
        print(f"\n==== 初步模型评估 ====")
        print(f"初始运动准确率: {motion_accuracy * 100:.2f}%")
        print(f"初始静止准确率: {static_accuracy * 100:.2f}%")
        
        if motion_accuracy < 0.5 and static_accuracy < 0.5:
            # 如果大多数预测错误，那么可能需要反转预测结果
            print("特征与标签关系相反，反转预测...")
            self.invert_prediction = True
            motion_correct = np.sum(motion_predictions == 0)
            static_correct = np.sum(static_predictions == 1)
        else:
            self.invert_prediction = False
        
        total_correct = motion_correct + static_correct
        total_samples = len(motion_predictions) + len(static_predictions)
        accuracy = total_correct / total_samples if total_samples > 0 else 0
        
        print(f"\n==== 最终模型评估 ====")
        print(f"训练集总体准确率: {accuracy * 100:.2f}%")
        print(f"运动样本正确率: {motion_correct / len(motion_predictions) * 100:.2f}% (总数: {len(motion_predictions)})")
        print(f"静止样本正确率: {static_correct / len(static_predictions) * 100:.2f}% (总数: {len(static_predictions)})")
        
        # 可视化特征分布和决策阈值
        if visualize:
            self._visualize_decision_boundary(motion_values, static_values, self.threshold, 
                                            self.feature_names[best_feature_idx])
            
            # 可视化预测结果
            y_true = np.concatenate([np.ones(len(motion_values)), np.zeros(len(static_values))])
            y_pred = np.concatenate([motion_predictions, static_predictions])
            if self.invert_prediction:
                y_pred = 1 - y_pred
            
            self._visualize_confusion_matrix(y_true, y_pred)
        
        # 保存模型
        try:
            model_data = {
                'threshold': self.threshold,
                'best_feature_idx': self.best_feature_idx,
                'invert_prediction': self.invert_prediction,
                'feature_name': self.feature_names[best_feature_idx]
            }
            
            # 创建输出目录
            output_dir = 'motion_output'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
                
            with open(os.path.join(output_dir, 'motion_detection_model.pkl'), 'wb') as f:
                pickle.dump(model_data, f)
            
            print(f"\n模型已保存到 {output_dir}/motion_detection_model.pkl")
            print(f"模型使用特征: {self.feature_names[best_feature_idx]}")
            print(f"决策阈值: {self.threshold:.4f}")
            print(f"预测方向: {'反向' if self.invert_prediction else '正向'}")
            
            # 保存模型摘要到文本文件
            with open(os.path.join(output_dir, 'model_summary.txt'), 'w', encoding='utf-8') as f:
                f.write("===== 运动检测模型摘要 =====\n\n")
                f.write(f"使用特征: {self.feature_names[best_feature_idx]}\n")
                f.write(f"决策阈值: {self.threshold:.4f}\n")
                f.write(f"预测方向: {'反向' if self.invert_prediction else '正向'}\n\n")
                f.write("特征均值比较:\n")
                for i, name in enumerate(self.feature_names):
                    f.write(f"{name}: 运动={motion_mean[i]:.4f}, 静止={static_mean[i]:.4f}, 差异={feature_diff[i]:.4f}\n")
                f.write(f"\n训练集总体准确率: {accuracy * 100:.2f}%\n")
                f.write(f"运动样本正确率: {motion_correct / len(motion_predictions) * 100:.2f}% (总数: {len(motion_predictions)})\n")
                f.write(f"静止样本正确率: {static_correct / len(static_predictions) * 100:.2f}% (总数: {len(static_predictions)})\n")
            
            print(f"模型摘要已保存到 {output_dir}/model_summary.txt")
        except Exception as e:
            print(f"保存模型失败: {str(e)}")
    
    def _visualize_all_features_distribution(self, motion_features, static_features):
        """可视化所有特征的分布对比"""
        try:
            plt.figure(figsize=(18, 15))
            
            # 为每个特征创建箱线图对比
            for i, feature_name in enumerate(self.feature_names):
                motion_vals = motion_features[:, i]
                static_vals = static_features[:, i]
                
                # 创建子图
                plt.subplot(3, 3, i+1)
                
                # 绘制箱线图
                box_data = [motion_vals, static_vals]
                box = plt.boxplot(box_data, patch_artist=True)
                
                # 设置箱线图颜色
                colors = ['lightblue', 'lightgreen']
                for patch, color in zip(box['boxes'], colors):
                    patch.set_facecolor(color)
                
                plt.xticks([1, 2], ['运动', '静止'])
                plt.title(feature_name)
                plt.ylabel('特征值')
                plt.grid(True, linestyle='--', alpha=0.7)
                
                # 添加均值点
                plt.plot(1, np.mean(motion_vals), 'o', color='red', markersize=8)
                plt.plot(2, np.mean(static_vals), 'o', color='red', markersize=8)
                
                # 添加差异值
                diff = np.abs(np.mean(motion_vals) - np.mean(static_vals))
                plt.text(1.5, max(np.mean(motion_vals), np.mean(static_vals)) * 1.1, 
                        f'差异: {diff:.4f}', horizontalalignment='center')
            
            plt.tight_layout()
            viz_dir = 'motion_output/visualization'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, 'feature_distributions.png'))
            plt.close()
            print("已保存所有特征分布对比图: feature_distributions.png")
            
            # 添加PCA可视化
            self._visualize_pca(motion_features, static_features)
            
        except Exception as e:
            print(f"可视化所有特征分布失败: {str(e)}")
    
    def _visualize_pca(self, motion_features, static_features):
        """使用PCA可视化特征"""
        try:
            # 组合特征
            X = np.vstack([motion_features, static_features])
            y = np.concatenate([np.ones(len(motion_features)), np.zeros(len(static_features))])
            
            # 应用PCA
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X)
            
            plt.figure(figsize=(10, 8))
            
            # 绘制散点图
            scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='coolwarm', 
                                 alpha=0.8, s=100, edgecolors='k')
            
            plt.xlabel(f'主成分1 (方差占比: {pca.explained_variance_ratio_[0]:.2f})')
            plt.ylabel(f'主成分2 (方差占比: {pca.explained_variance_ratio_[1]:.2f})')
            plt.title('PCA降维后的特征分布')
            plt.colorbar(scatter, label='类别 (1=运动, 0=静止)')
            plt.grid(True, linestyle='--', alpha=0.7)
            
            # 为每个类别计算质心
            motion_center = np.mean(X_pca[y==1], axis=0)
            static_center = np.mean(X_pca[y==0], axis=0)
            
            # 绘制质心
            plt.plot(motion_center[0], motion_center[1], 'o', markersize=15, 
                    markerfacecolor='none', markeredgecolor='blue', markeredgewidth=3, label='运动质心')
            plt.plot(static_center[0], static_center[1], 'o', markersize=15, 
                    markerfacecolor='none', markeredgecolor='red', markeredgewidth=3, label='静止质心')
            
            # 绘制连接线
            plt.plot([motion_center[0], static_center[0]], [motion_center[1], static_center[1]], 
                    'k--', linewidth=2, label='类别分离线')
            
            plt.legend()
            plt.tight_layout()
            
            viz_dir = 'motion_output/visualization'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, 'pca_visualization.png'))
            plt.close()
            print("已保存PCA特征可视化: pca_visualization.png")
            
        except Exception as e:
            print(f"PCA可视化失败: {str(e)}")
    
    def _visualize_decision_boundary(self, motion_values, static_values, threshold, feature_name):
        """可视化决策边界和特征分布"""
        try:
            plt.figure(figsize=(12, 7))
            
            # 绘制直方图
            bins = 30
            plt.hist(motion_values, bins=bins, alpha=0.6, color='orange', label='运动', density=True)
            plt.hist(static_values, bins=bins, alpha=0.6, color='blue', label='静止', density=True)
            
            # 添加核密度估计
            try:
                from scipy.stats import gaussian_kde
                
                motion_kde = gaussian_kde(motion_values)
                static_kde = gaussian_kde(static_values)
                
                x_min = min(np.min(motion_values), np.min(static_values))
                x_max = max(np.max(motion_values), np.max(static_values))
                
                x = np.linspace(x_min, x_max, 1000)
                plt.plot(x, motion_kde(x), 'orange', linewidth=2)
                plt.plot(x, static_kde(x), 'blue', linewidth=2)
            except Exception as e:
                print(f"核密度估计绘制失败: {str(e)}")
            
            # 添加决策阈值
            ymin, ymax = plt.ylim()
            plt.vlines(x=threshold, ymin=ymin, ymax=ymax, 
                     colors='r', linestyles='dashed', linewidth=2, label=f'阈值 ({threshold:.4f})')
            
            # 添加注释
            plt.text(threshold + (x_max - x_min) * 0.05, ymax * 0.9, '运动区域', 
                    color='orange', fontsize=12, ha='left')
            plt.text(threshold - (x_max - x_min) * 0.05, ymax * 0.9, '静止区域', 
                    color='blue', fontsize=12, ha='right')
            
            plt.title(f'特征分布与决策边界\n使用特征: {feature_name}')
            plt.xlabel(f'{feature_name} 特征值')
            plt.ylabel('归一化密度')
            plt.legend()
            
            # 添加描述性文本
            motion_mean = np.mean(motion_values)
            static_mean = np.mean(static_values)
            motion_std = np.std(motion_values)
            static_std = np.std(static_values)
            
            textstr = (f'运动均值: {motion_mean:.4f}\n'
                     f'静止均值: {static_mean:.4f}\n'
                     f'运动标准差: {motion_std:.4f}\n'
                     f'静止标准差: {static_std:.4f}\n'
                     f'阈值: {threshold:.4f}')
            
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, 
                    fontsize=10, verticalalignment='top', bbox=props)
            
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.tight_layout()
            
            viz_dir = 'motion_output/visualization'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, 'decision_boundary.png'))
            plt.close()
            print("已保存决策边界可视化: decision_boundary.png")
        except Exception as e:
            print(f"可视化决策边界失败: {str(e)}")
    
    def _visualize_confusion_matrix(self, y_true, y_pred):
        """可视化混淆矩阵"""
        try:
            cm = confusion_matrix(y_true, y_pred)
            plt.figure(figsize=(8, 6))
            
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                      xticklabels=['静止', '运动'],
                      yticklabels=['静止', '运动'])
            
            plt.title('混淆矩阵')
            plt.ylabel('真实类别')
            plt.xlabel('预测类别')
            
            # 添加分类报告文本
            report = classification_report(y_true, y_pred, target_names=['静止', '运动'])
            
            # 保存分类报告到文本文件
            output_dir = 'motion_output'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
                
            with open(os.path.join(output_dir, 'classification_report.txt'), 'w', encoding='utf-8') as f:
                f.write(report)
            
            plt.tight_layout()
            
            viz_dir = 'motion_output/visualization'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, 'confusion_matrix.png'))
            plt.close()
            print("已保存混淆矩阵可视化: confusion_matrix.png")
            print(f"分类报告已保存到 {output_dir}/classification_report.txt")
        except Exception as e:
            print(f"可视化混淆矩阵失败: {str(e)}")
    
    def predict(self, csi_data, visualize=False, filename='unknown'):
        """
        预测是否有运动
        """
        preprocessed_data = self.preprocess_csi(csi_data, visualize=visualize, filename=filename)
        if preprocessed_data is None:
            return None
            
        features = self.extract_features(preprocessed_data, visualize=visualize, filename=filename)
        
        if features is None:
            return None
        
        # 使用已训练的阈值进行预测
        feature_value = features[self.best_feature_idx]
        raw_prediction = feature_value > self.threshold
        
        # 如果需要反转预测结果
        if hasattr(self, 'invert_prediction') and self.invert_prediction:
            prediction = 0 if raw_prediction else 1
        else:
            prediction = 1 if raw_prediction else 0
        
        # 可视化预测过程
        if visualize:
            self._visualize_prediction(feature_value, self.threshold, prediction, self.feature_names[self.best_feature_idx], filename)
        
        return prediction
    
    def _visualize_prediction(self, feature_value, threshold, prediction, feature_name, filename):
        """可视化预测过程"""
        try:
            plt.figure(figsize=(10, 6))
            
            # 创建水平条形图显示特征值
            plt.barh(['特征值'], [feature_value], color='skyblue')
            
            # 添加阈值线
            plt.axvline(x=threshold, color='r', linestyle='--', label=f'阈值 ({threshold:.4f})')
            
            # 标记预测结果
            result_text = '有运动' if prediction == 1 else '无运动'
            plt.text(feature_value, 0, f" {feature_value:.4f}", 
                    verticalalignment='center', fontsize=12)
            
            # 添加背景色区分区域
            plt.axvspan(threshold, plt.xlim()[1], facecolor='salmon', alpha=0.2, label='运动区域')
            plt.axvspan(plt.xlim()[0], threshold, facecolor='lightgreen', alpha=0.2, label='静止区域')
            
            plt.title(f'预测结果: {result_text}\n使用特征: {feature_name}')
            plt.xlabel('特征值')
            plt.yticks([])
            plt.legend()
            
            # 添加预测信息文本框
            info_text = (f'文件: {filename}\n'
                       f'特征: {feature_name}\n'
                       f'特征值: {feature_value:.4f}\n'
                       f'阈值: {threshold:.4f}\n'
                       f'预测: {result_text}')
            
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            plt.text(0.05, 0.1, info_text, transform=plt.gca().transAxes, 
                    fontsize=10, verticalalignment='bottom', bbox=props)
            
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.tight_layout()
            
            viz_dir = 'motion_output/visualization/predictions'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, f'prediction_{filename}.png'))
            plt.close()
            print(f"已保存预测可视化: prediction_{filename}.png")
        except Exception as e:
            print(f"可视化预测失败: {str(e)}")
    
    def load_model(self, model_path='motion_output/motion_detection_model.pkl'):
        """
        加载训练好的模型
        """
        try:
            with open(model_path, 'rb') as f:
                model_data = pickle.load(f)
                
            self.threshold = model_data['threshold']
            self.best_feature_idx = model_data['best_feature_idx']
            self.invert_prediction = model_data['invert_prediction']
            
            feature_name = model_data.get('feature_name', self.feature_names[self.best_feature_idx])
            
            print("\n==== 模型加载成功 ====")
            print(f"特征: {feature_name}")
            print(f"阈值: {self.threshold:.4f}")
            print(f"预测方向: {'反向' if self.invert_prediction else '正向'}")
            return True
        except Exception as e:
            print(f"加载模型失败: {str(e)}")
            return False
    
    def evaluate(self, test_dir, labels, visualize=True):
        """
        评估模型在测试集上的表现
        """
        predictions = []
        files = []
        
        print("\n==== 开始测试集评估 ====")
        
        # 创建每个样本的特征值记录，用于可视化
        feature_values = []
        
        for file in sorted(os.listdir(test_dir)):
            if file.endswith('.csv'):
                file_path = os.path.join(test_dir, file)
                print(f"处理测试文件: {file}")
                
                csi_data = self.load_csi_data(file_path, visualize=False)
                
                if csi_data is not None:
                    # 获取预处理数据和特征
                    preprocessed_data = self.preprocess_csi(csi_data, visualize=False)
                    features = self.extract_features(preprocessed_data, visualize=False)
                    
                    if features is not None:
                        # 记录关键特征值
                        feature_values.append(features[self.best_feature_idx])
                        
                        # 获取预测结果
                        prediction = self.predict(csi_data, visualize=visualize, filename=file)
                        
                        if prediction is not None:
                            predictions.append(prediction)
                            files.append(file)
                            print(f"预测结果: {'有运动' if prediction == 1 else '无运动'}")
                        else:
                            print(f"无法预测文件: {file}")
                    else:
                        print(f"无法提取特征: {file}")
                else:
                    print(f"无法加载测试文件: {file}")
        
        if len(predictions) == 0:
            print("错误: 没有得到任何预测结果!")
            return None, predictions, files
        
        if len(labels) != len(predictions):
            print(f"警告: 标签数量 ({len(labels)}) 与预测数量 ({len(predictions)}) 不匹配")
        else:
            # 转换标签为整数
            int_labels = [int(label) for label in labels]
            
            # 计算准确率
            accuracy = accuracy_score(int_labels, predictions)
            print(f"\n测试集准确率: {accuracy * 100:.2f}%")
            
            # 可视化测试结果
            if visualize:
                self._visualize_test_results(files, feature_values, predictions, int_labels, self.threshold)
                
                # 绘制混淆矩阵
                cm = confusion_matrix(int_labels, predictions)
                plt.figure(figsize=(8, 6))
                
                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                          xticklabels=['静止', '运动'],
                          yticklabels=['静止', '运动'])
                
                plt.title('测试集混淆矩阵')
                plt.ylabel('真实类别')
                plt.xlabel('预测类别')
                
                viz_dir = 'motion_output/visualization'
                if not os.path.exists(viz_dir):
                    os.makedirs(viz_dir)
                plt.savefig(os.path.join(viz_dir, 'test_confusion_matrix.png'))
                plt.close()
                print("已保存测试集混淆矩阵可视化: test_confusion_matrix.png")
                
                # 生成分类报告
                report = classification_report(int_labels, predictions, target_names=['静止', '运动'])
                print("\n分类报告:")
                print(report)
                
                # 保存分类报告到文本文件
                output_dir = 'motion_output'
                with open(os.path.join(output_dir, 'test_classification_report.txt'), 'w', encoding='utf-8') as f:
                    f.write(report)
                print(f"测试集分类报告已保存到 {output_dir}/test_classification_report.txt")
        
        return accuracy, predictions, files
    
    def _visualize_test_results(self, files, feature_values, predictions, labels, threshold):
        """可视化测试结果"""
        try:
            plt.figure(figsize=(14, 8))
            
            # 创建散点图
            correct_indices = [i for i, (p, l) in enumerate(zip(predictions, labels)) if p == l]
            wrong_indices = [i for i, (p, l) in enumerate(zip(predictions, labels)) if p != l]
            
            # 绘制正确和错误的预测
            if correct_indices:
                plt.scatter([i for i in correct_indices], [feature_values[i] for i in correct_indices], 
                          marker='o', s=100, color='green', label='正确预测', alpha=0.7)
            
            if wrong_indices:
                plt.scatter([i for i in wrong_indices], [feature_values[i] for i in wrong_indices], 
                          marker='x', s=100, color='red', label='错误预测', alpha=0.7)
            
            # 添加阈值线
            plt.axhline(y=threshold, color='k', linestyle='--', linewidth=2, label=f'阈值 ({threshold:.4f})')
            
            # 标记区域
            plt.axhspan(threshold, max(feature_values) * 1.1, facecolor='salmon', alpha=0.1, label='运动区域')
            plt.axhspan(min(feature_values) * 0.9, threshold, facecolor='lightgreen', alpha=0.1, label='静止区域')
            
            # 添加文件名标签（每5个点显示一个）
            for i in range(0, len(files), 5):
                plt.text(i, feature_values[i], files[i], rotation=45, fontsize=8)
            
            # 添加类别标记
            for i, (file, pred, label) in enumerate(zip(files, predictions, labels)):
                color = 'green' if pred == label else 'red'
                marker = 'o' if pred == 1 else 's'
                plt.plot(i, feature_values[i] * 0.98, marker, color=color, markersize=8)
            
            plt.title('测试集预测结果可视化')
            plt.xlabel('样本索引')
            plt.ylabel(f'特征值 ({self.feature_names[self.best_feature_idx]})')
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.legend()
            
            # 添加统计信息
            accuracy = sum(1 for p, l in zip(predictions, labels) if p == l) / len(predictions)
            num_motion = sum(1 for l in labels if l == 1)
            num_static = sum(1 for l in labels if l == 0)
            motion_acc = sum(1 for p, l in zip(predictions, labels) if p == l and l == 1) / num_motion if num_motion > 0 else 0
            static_acc = sum(1 for p, l in zip(predictions, labels) if p == l and l == 0) / num_static if num_static > 0 else 0
            
            textstr = (f'总体准确率: {accuracy:.2f}\n'
                     f'运动样本数: {num_motion}\n'
                     f'静止样本数: {num_static}\n'
                     f'运动准确率: {motion_acc:.2f}\n'
                     f'静止准确率: {static_acc:.2f}')
            
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            plt.text(0.02, 0.97, textstr, transform=plt.gca().transAxes, 
                    fontsize=10, verticalalignment='top', bbox=props)
            
            plt.tight_layout()
            
            viz_dir = 'motion_output/visualization'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, 'test_results.png'))
            plt.close()
            print("已保存测试结果可视化: test_results.png")
            
            # 绘制条形图
            self._visualize_test_bar_chart(files, predictions, labels)
            
        except Exception as e:
            print(f"可视化测试结果失败: {str(e)}")
    
    def _visualize_test_bar_chart(self, files, predictions, labels):
        """绘制测试结果条形图"""
        try:
            plt.figure(figsize=(15, 8))
            
            # 使用简化的文件名
            short_files = [f.split('.')[0] for f in files]
            
            # 为每个样本设置颜色
            colors = ['red' if pred != label else 'green' for pred, label in zip(predictions, labels)]
            
            # 绘制条形图，高度为1表示运动，0表示静止
            plt.bar(range(len(short_files)), predictions, color=colors, alpha=0.7)
            
            # 绘制真实标签轮廓
            for i, label in enumerate(labels):
                plt.plot([i-0.4, i+0.4], [label, label], 'k-', linewidth=2)
            
            # 设置刻度和标签
            plt.xticks(range(len(short_files)), short_files, rotation=90, fontsize=8)
            plt.yticks([0, 1], ['静止', '运动'])
            plt.xlabel('样本文件')
            plt.ylabel('预测结果')
            plt.title('测试集预测结果对比\n(绿色表示正确预测，红色表示错误预测，黑线表示真实标签)')
            
            # 添加图例
            from matplotlib.patches import Patch
            legend_elements = [
                Patch(facecolor='green', alpha=0.7, label='正确预测'),
                Patch(facecolor='red', alpha=0.7, label='错误预测'),
                Patch(facecolor='white', edgecolor='black', label='真实标签')
            ]
            plt.legend(handles=legend_elements)
            
            plt.tight_layout()
            
            viz_dir = 'motion_output/visualization'
            if not os.path.exists(viz_dir):
                os.makedirs(viz_dir)
            plt.savefig(os.path.join(viz_dir, 'test_bar_chart.png'))
            plt.close()
            print("已保存测试条形图: test_bar_chart.png")
            
        except Exception as e:
            print(f"绘制测试条形图失败: {str(e)}")


def main():
    print("=== CSI 运动检测程序启动 ===")
    
    # 创建主输出目录
    output_dir = 'motion_output'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 创建可视化目录
    viz_dir = os.path.join(output_dir, 'visualization')
    if not os.path.exists(viz_dir):
        os.makedirs(viz_dir)
    
    # 设置数据路径
    base_dir = "../benchmark/motion_detection"
    motion_dir = os.path.join(base_dir, "evaluation_motion")
    static_dir = os.path.join(base_dir, "evaluation_static")
    test_dir = os.path.join(base_dir, "test")
    
    # 确保目录存在
    if not os.path.exists(motion_dir):
        print(f"警告: 目录不存在 {motion_dir}")
        # 允许用户输入正确路径
        motion_dir = input("请输入运动数据目录路径: ")
    
    if not os.path.exists(static_dir):
        print(f"警告: 目录不存在 {static_dir}")
        static_dir = input("请输入静止数据目录路径: ")
    
    if not os.path.exists(test_dir):
        print(f"警告: 目录不存在 {test_dir}")
        test_dir = input("请输入测试数据目录路径: ")
    
    # 检查model.pkl是否存在
    model_exists = os.path.exists(os.path.join(output_dir, 'motion_detection_model.pkl'))
    
    detector = MotionDetection()
    
    # 可视化设置
    visualize = True
    if visualize:
        print(f"已启用可视化功能，将生成图表到{output_dir}/visualization目录")
    
    # 根据用户选择决定是训练还是加载模型
    if model_exists:
        choice = input("发现已有模型，是否重新训练? (y/n): ")
        if choice.lower() == 'y':
            print("开始训练模型...")
            detector.train(motion_dir, static_dir, visualize=visualize)
        else:
            print("加载已有模型...")
            if not detector.load_model():
                print("加载模型失败，将重新训练...")
                detector.train(motion_dir, static_dir, visualize=visualize)
    else:
        print("未发现已有模型，开始训练...")
        detector.train(motion_dir, static_dir, visualize=visualize)
    
    # 处理测试集数据，并输出预测结果
    print("\n处理测试集数据...")
    test_results = []
    
    try:
        test_files = sorted(os.listdir(test_dir))
        print(f"发现 {len(test_files)} 个测试文件")
        
        for file in test_files:
            if file.endswith('.csv'):
                file_path = os.path.join(test_dir, file)
                print(f"处理测试文件: {file}")
                
                csi_data = detector.load_csi_data(file_path, visualize=False)
                if csi_data is not None:
                    prediction = detector.predict(csi_data, visualize=visualize, filename=file)
                    
                    if prediction is not None:
                        test_results.append((file, prediction))
                        print(f"预测结果: {'有运动' if prediction == 1 else '无运动'}")
                    else:
                        print(f"无法预测文件: {file}")
                else:
                    print(f"无法加载测试文件: {file}")
        
        # 保存测试结果
        try:
            with open(os.path.join(output_dir, 'test_predictions.txt'), 'w', encoding='utf-8') as f:
                for file, pred in test_results:
                    f.write(f"{file}, {pred}\n")
            
            print(f"\n预测完成，共处理 {len(test_results)} 个文件，结果已保存到 {output_dir}/test_predictions.txt")
            
            # 生成HTML报告
            generate_html_report(test_results, detector)
            
        except Exception as e:
            print(f"保存预测结果失败: {str(e)}")
    except Exception as e:
        print(f"处理测试数据时出错: {str(e)}")

def generate_html_report(test_results, detector):
    """生成HTML格式的报告"""
    try:
        html_content = """
        <!DOCTYPE html>
        <html>
        <head>
            <meta charset="UTF-8">
            <title>CSI运动检测结果报告</title>
            <style>
                body { font-family: Arial, "Microsoft YaHei", sans-serif; margin: 20px; }
                h1 { color: #2c3e50; }
                h2 { color: #3498db; }
                .container { max-width: 1200px; margin: 0 auto; }
                table { width: 100%; border-collapse: collapse; margin: 20px 0; }
                th, td { padding: 12px; text-align: left; border-bottom: 1px solid #ddd; }
                th { background-color: #f5f5f5; }
                tr:hover { background-color: #f9f9f9; }
                .motion { color: #e74c3c; font-weight: bold; }
                .static { color: #27ae60; font-weight: bold; }
                .summary { background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px; }
                .footer { margin-top: 30px; text-align: center; color: #7f8c8d; font-size: 0.8em; }
                .charts { display: flex; flex-wrap: wrap; justify-content: space-around; margin: 20px 0; }
                .chart { margin: 10px; text-align: center; }
                .chart img { max-width: 100%; height: auto; border: 1px solid #ddd; border-radius: 4px; }
            </style>
        </head>
        <body>
            <div class="container">
                <h1>CSI运动检测结果报告</h1>
                <div class="summary">
                    <h2>摘要</h2>
                    <p>使用特征: """ + detector.feature_names[detector.best_feature_idx] + """</p>
                    <p>决策阈值: """ + f"{detector.threshold:.4f}" + """</p>
                    <p>总测试样本数: """ + str(len(test_results)) + """</p>
                    <p>运动样本数: """ + str(sum(1 for _, pred in test_results if pred == 1)) + """</p>
                    <p>静止样本数: """ + str(sum(1 for _, pred in test_results if pred == 0)) + """</p>
                </div>
                
                <h2>测试结果详情</h2>
                <table>
                    <tr>
                        <th>序号</th>
                        <th>文件名</th>
                        <th>预测结果</th>
                    </tr>
        """
        
        # 添加表格行
        for i, (file, pred) in enumerate(test_results, 1):
            result_class = "motion" if pred == 1 else "static"
            result_text = "有运动" if pred == 1 else "无运动"
            
            html_content += f"""
                    <tr>
                        <td>{i}</td>
                        <td>{file}</td>
                        <td class="{result_class}">{result_text}</td>
                    </tr>
            """
        
        # 添加可视化图表
        html_content += """
                </table>
                
                <h2>可视化结果</h2>
                <div class="charts">
        """
        
        # 添加主要可视化图表
        viz_files = [
            {"file": "feature_distributions.png", "title": "特征分布对比"},
            {"file": "decision_boundary.png", "title": "决策阈值与特征分布"},
            {"file": "pca_visualization.png", "title": "PCA特征可视化"},
            {"file": "confusion_matrix.png", "title": "混淆矩阵"},
            {"file": "test_results.png", "title": "测试集预测结果"},
            {"file": "test_bar_chart.png", "title": "测试集预测结果条形图"}
        ]
        
        for viz in viz_files:
            viz_path = os.path.join("motion_output/visualization", viz["file"])
            if os.path.exists(viz_path):
                html_content += f"""
                    <div class="chart">
                        <h3>{viz["title"]}</h3>
                        <img src="visualization/{viz["file"]}" alt="{viz["title"]}">
                    </div>
                """
        
        # 结束HTML
        html_content += """
                </div>
                
                <div class="footer">
                    <p>生成时间: """ + pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S") + """</p>
                    <p>CSI运动检测系统 - 基于Wi-Fi信号的人体运动感知</p>
                </div>
            </div>
        </body>
        </html>
        """
        
        # 写入HTML文件
        output_dir = 'motion_output'
        with open(os.path.join(output_dir, "motion_detection_report.html"), "w", encoding="utf-8") as f:
            f.write(html_content)
            
        print(f"已生成HTML报告: {output_dir}/motion_detection_report.html")
    except Exception as e:
        print(f"生成HTML报告失败: {str(e)}")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"程序异常终止: {str(e)}")

=== CSI 运动检测程序启动 ===
已启用可视化功能，将生成图表到motion_output/visualization目录
未发现已有模型，开始训练...

==== 开始模型训练 ====
开始加载运动和静止数据...
发现 21 个运动数据文件
处理运动文件: CSI_20250220_204932.csv
警告: 特征包含NaN或无穷大值，将被替换
成功提取特征: CSI_20250220_204932.csv
处理运动文件: CSI_20250220_205301.csv
警告: 特征包含NaN或无穷大值，将被替换
成功提取特征: CSI_20250220_205301.csv
处理运动文件: CSI_20250220_205318.csv
警告: 特征包含NaN或无穷大值，将被替换
成功提取特征: CSI_20250220_205318.csv
处理运动文件: CSI_20250220_205330.csv
已保存原始CSI可视化: raw_csi_CSI_20250220_205330.csv.png
已保存预处理可视化: preprocess_CSI_20250220_205330.csv.png
警告: 特征包含NaN或无穷大值，将被替换
已保存特征可视化: features_CSI_20250220_205330.csv.png
成功提取特征: CSI_20250220_205330.csv
处理运动文件: CSI_20250220_205343.csv
警告: 特征包含NaN或无穷大值，将被替换
成功提取特征: CSI_20250220_205343.csv
处理运动文件: CSI_20250220_205356.csv
警告: 特征包含NaN或无穷大值，将被替换
成功提取特征: CSI_20250220_205356.csv
处理运动文件: CSI_20250220_205406.csv
警告: 特征包含NaN或无穷大值，将被替换
成功提取特征: CSI_20250220_205406.csv
处理运动文件: CSI_20250220_205439.csv
警告: 特征包含NaN或无穷大值，将被替换
成功提取特征: CSI_20250220_205439.csv
处理运动文件: CSI_20250220_205451.csv
警告: 特